### Tutorial on how to use the train/test method listed in the s2s `traintest` module

For cross-validation, we choose the train/test splitting method in the s2s `time` module and get the split groups.

We start by importing the required libraries and generating an example `AdventCalendar`.

In [1]:
import s2s.time
import s2s.traintest

In [2]:
calendar = s2s.time.AdventCalendar(anchor_date=(10, 15), freq='180d')
intervals = calendar.map_years(2019, 2021, flat=True)
intervals

0    (2021-04-18, 2021-10-15]
1    (2020-10-20, 2021-04-18]
2    (2020-04-18, 2020-10-15]
3    (2019-10-21, 2020-04-18]
4    (2019-04-18, 2019-10-15]
5    (2018-10-20, 2019-04-18]
dtype: interval

Example of `kfold` method.

In [5]:
calendar.set_traintest_method("kfold", n_splits = 2)

Get train/test groups.

In [7]:
traintest_group = calendar.get_traintest()
traintest_group

,anchor_year,i_intervals,intervals,fold_0,fold_1
0,2021,0,"(2021-04-18, 2021-10-15]",train,test
1,2021,1,"(2020-10-20, 2021-04-18]",train,test
2,2020,0,"(2020-04-18, 2020-10-15]",test,train
3,2020,1,"(2019-10-21, 2020-04-18]",test,train
4,2019,0,"(2019-04-18, 2019-10-15]",test,train
5,2019,1,"(2018-10-20, 2019-04-18]",test,train
